In [ ]:
!pip uninstall tensorflow-estimator
!pip install tensorflow-estimator
!pip uninstall keras
!pip install keras

Found existing installation: tensorflow-estimator 2.8.0
Uninstalling tensorflow-estimator-2.8.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/tensorflow_estimator-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow_estimator/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-estimator-2.8.0
     |████████████████████████████████| 462 kB 4.0 MB/s 
26.0
Found existing installation: keras 2.8.0
Uninstalling keras-2.8.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras/*
Proceed (y/n)? y
  Successfully uninstalled keras-2.8.0
     |████████████████████████████████| 1.4 MB 4.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [ ]:
import tensorflow
import threading

# baseline cnn model for mnist
import numpy as np
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
import timeit



scores = np.array([[0.0 for i in range(5)] for j in range(11)])
histories = np.array([[Sequential() for i in range(5)] for j in range(11)])
models = np.array([[Sequential() for i in range(5)] for j in range(11)])
bPredicts = np.array([[[0 for i in range(10)] for k in range(10002)] for j in range(5)])
cPredicts = np.array([[0 for i in range(10002)] for j in range(5)])

def evaluate_model(dataX, dataY, layerNum, lossType, layer2Count, index, n_folds=5):
	global scores
	global histories
	global models
	scorez, historiez, modelz = list(), list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		print(index)
		model = define_model(layerNum, lossType, layer2Count)
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		modelz.append(model)
		scorez.append(acc)
		historiez.append(history)
	for i in range(5):
		scores[index][i] = scorez[i]
		histories[index][i] = historiez[i]
		models[index][i] = modelz[i]
	return

def evaluate_model_parallel(dataX, dataY, layerNum, lossType, layer2Count, index, n_folds=5):
	global scores
	global histories
	global models
	scorez, historiez, modelz = list(), list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		print(index)
		model = define_model(layerNum, lossType, layer2Count)
		model = multi_gpu_model(model,gpus=2)
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		modelz.append(model)
		scorez.append(acc)
		historiez.append(history)
	for i in range(5):
		scores[index][i] = scorez[i]
		histories[index][i] = historiez[i]
		models[index][i] = modelz[i]
	return


def define_model(layerNum, lossType, layer2Count):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(layer2Count, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(layerNum, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])
	return model

def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		plt.subplot(2, 1, 1)
		plt.title('Cross Entropy Loss')
		plt.plot(histories[i].history['loss'], color='blue', label='train')
		plt.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		plt.subplot(2, 1, 2)
		plt.title('Classification Accuracy')
		plt.plot(histories[i].history['accuracy'], color='blue', label='train')
		plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	plt.show()
 
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	plt.boxplot(scores)
	plt.show()

def predict_and_print(img,imgy,num):
  global bPredicts
  global cPredicts
  for i in range(5):
    print(num)
    if(num < 10):
      bPredicts[i][0][num] = timeit.default_timer()
    else:
      cPredicts[i][0] = timeit.default_timer()
    model = load_model('/content/drive/MyDrive/PPmodels/PPmodel'+str(num+1)+'-'+str(i)+'.h5')
    for j in range(len(img)):
      if j%100 == 0:
        print(j/100)
      temp = img[j].reshape(1,28,28,1)
      if(num != 10):
        bPredicts[i][j+1][num] = np.argmax(model.predict(temp))
      else:
        cPredicts[i][j+1] = np.argmax(model.predict(temp))
    if(num != 10):
      bPredicts[i][len(img)+1][num] = timeit.default_timer()
    else:
      cPredicts[i][len(img)] = timeit.default_timer()
    #print(xTest[0])
  return

 
__name__ == '__main__'
global scores
global histories
global bPredicts
global cPredicts
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
    
sizes = [0 for i in range(10)]
yTrain = to_categorical(yTrain)
yTest = to_categorical(yTest)
yTrainSep = np.array([[[0 for j in range(2)] for i in range(len(yTrain))] for k in range(10)], dtype = 'float32')
yTestSep = np.array([[[0 for j in range(2)] for i in range(len(yTest))] for k in range(10)], dtype = 'float32')
for j in range(10):
  for i in range(len(yTrain)):
    if(yTrain[i][j] == 0):
      yTrainSep[j][i][1] = 1
    else:
      yTrainSep[j][i][0] = 1
for j in range(10):
  for i in range(len(yTest)):
    if(yTest[i][j] == 0):
      yTestSep[j][i][1] = 1
    else:
      yTestSep[j][i][0] = 1
   
#modelSep = []
#for i in range(10):
#  modelSep.append(define_model(2,'binary_crossentropy',100))
#modelSep.append(define_model(10,'categorical_crossentropy',100))

xTrain = xTrain.reshape((xTrain.shape[0], 28, 28, 1))
xTest = xTest.reshape((xTest.shape[0], 28, 28, 1))
#for i in range(len(xTest)):
  #xTest[i] = xTest[i].reshape(28,28,1)
print(xTest[0].shape)
xTrain, xTest = prep_pixels(xTrain, xTest)
testImage = xTest[0]
testImage = testImage.reshape(28,28,1)
#print('PPmodel'+str(0)+'-'+str(1)+'.h5')
t = [threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[0],2,'binary_crossentropy',10,0)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[1],2,'binary_crossentropy',10,1)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[2],2,'binary_crossentropy',10,2)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[3],2,'binary_crossentropy',100,3)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[4],2,'binary_crossentropy',100,4)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[5],2,'binary_crossentropy',100,5)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[6],2,'binary_crossentropy',100,6)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[7],2,'binary_crossentropy',100,7)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[8],2,'binary_crossentropy',100,8)),threading.Thread(target=evaluate_model, args=(xTrain, yTrainSep[9],2,'binary_crossentropy',100,9)),threading.Thread(target=evaluate_model, args=(xTrain, yTrain,10,'categorical_crossentropy',100,10))]
s = [threading.Thread(target=predict_and_print, args=(xTest,yTestSep[0],0)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[1],1)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[2],2)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[3],3)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[4],4)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[5],5)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[6],6)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[7],7)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[8],8)),threading.Thread(target=predict_and_print, args=(xTest,yTestSep[9],9)),threading.Thread(target=predict_and_print, args=(xTest,yTest,10))]
for i in range(10):
  #if(i > 5):
    evaluate_model(xTrain, yTrainSep[i],2,'binary_crossentropy',10,i)
    #for j in range(5):
      #models[i][j].save('PPmodel'+str(i)+'-'+str(j)+'.h5')
#evaluate_model(xTrain, yTrain,10,'categorical_crossentropy',100,10)
#for j in range(5):
  #models[10][j].save('PPmodel'+str(10)+'-'+str(j)+'.h5')
#evaluate_model(xTrain, yTrain,10,'categorical_crossentropy',100,10)
#for j in range(5):
#  models[10][j].save('PPmodel'+str(11)+'-'+str(j)+'.h5')

#summarize_diagnostics(histories[0])
#summarize_mance(scores[0])
#for i in range(10):
#  t[i].start()
#  t[i].join()

#for i in range(1):

#print(scores[0])
#print(np.argmax(models[0][0].predict(testImage)))
#print(xTest[0])
#for i in range(10):
  #s[i].start()
  #s[i].join()
  

#results = np.array([[[0 for k in range(3)] for i in range(2)] for j in range(5)])
#for i in range(5):
#  for j in range(10):
#    temp = bPredicts[i][len(yTest)+1][j] - bPredicts[i][0][j]
#    if temp > results[i][0][0]:
#      results[i][0][0] = temp
#  for j in range(len(yTest)):
#    for k in range(10):
#      counter = 0
#      if(bPredicts[i][j+1][k] == 0):
#        counter += 1
#      if(counter == 1):
#        if np.argmin(bPredicts[i][j+1]) == np.argmax(yTest[j]):
#          results[i][0][1] += 1
#        else:
#          results[i][0][2] += 1
#      else:
#        results[i][0][2] += 1
#predict_and_print(xTest,yTest,10)
#for i in range(5):
#  results[i][1][0] = cPredicts[i][len(yTest)+1] - cPredicts[i][0]
#  for j in range(len(yTest)):
#    if(cPredicts[i][j+1] == np.argmax(yTest[j])):
#      results[i][1][1] += 1
#    else:
#      results[i][1][2] += 2

#print(results)
